# R_DLGA algorithm _part2

## Overview

The performance of data-driven partial differential equations lacks stability when dealing with complex situations such as sparse data with high noise, so the robust deep learning Genetic algorithm (R-DLGA) is proposed.

## Technical path

The specific process of R-DLGA to solve this problem is as follows:

1. Run train.py to train the neural network, generate metadata and compute derivatives and obtain potential terms through genetic algorithms
2. Modify candidates in the train_pinn.py dict directory based on Step 1
3. Run train_pinn.py to get the final result

This part2 implements the functional code of the third step. After modifying the term calculation under src/util.py and the dict in part2 according to the result of the genetic algorithm in step 1, run the notebook to obtain the final result.

## Import code packages

In [ ]:
import os
import argparse
import random
import time

import numpy as np

from mindspore import nn, context, ops, set_seed
from mindspore import value_and_grad, jit, save_checkpoint
from mindspore import load_checkpoint, load_param_into_net
from mindspore.amp import DynamicLossScaler, auto_mixed_precision

from mindflow.utils import load_yaml_config
from mindflow.cell import MultiScaleFCSequential

The following `src` packages can be downloaded in [research/r_dlga/src] (https://gitee.com/mindspore/mindscience/tree/master/MindFlow/applications/research/r_dlga/src).

In [ ]:
from src import create_pinn_dataset, evaluate, cal_grads, cal_terms, pinn_loss_func
from src import get_dict_name, get_dicts, update_lib, calculate_coef, get_lefts

set_seed(0)
random.seed(0)

Parameters are configured, where --case has three choices, "burgers" means training on burgers equation, "cylinder_flow" means training on cylinder_flow dataset of navier_stokes2D equation, "periodic_hill" indicates training on a mountain flow dataset for the Reynolds mean Navier-Stokes equation.

In [ ]:
parser = argparse.ArgumentParser(description="train r_glda")
parser.add_argument("--case", type=str, default="burgers", choices=["burgers", "cylinder_flow", "periodic_hill"],
                    help="choose burgers, cylinder_flow or periodic_hill")
parser.add_argument("--mode", type=str, default="GRAPH", choices=["GRAPH", "PYNATIVE"],
                    help="Running in GRAPH_MODE OR PYNATIVE_MODE")
parser.add_argument("--device_target", type=str, default="Ascend", choices=["GPU", "Ascend"],
                    help="The target device to run, support 'Ascend', 'GPU'")
parser.add_argument("--device_id", type=int, default=0,
                    help="ID of the target device")
parser.add_argument("--config_file_path", type=str,
                    default="./configs/burgers.yaml")
input_args = parser.parse_args()

context.set_context(mode=context.GRAPH_MODE if input_args.mode.upper().startswith("GRAPH")
                    else context.PYNATIVE_MODE,
                    device_target=input_args.device_target,
                    device_id=input_args.device_id)
print(
    f"Running in {input_args.mode.upper()} mode, using device id: {input_args.device_id}.")
use_ascend = context.get_context(attr_key='device_target') == "Ascend"
print(use_ascend)
print("pid:", os.getpid())

Determine the equation to be trained and load the yaml file

In [ ]:
# get case name
case_name = input_args.case

# load configurations
config = load_yaml_config(input_args.config_file_path)

## Create a dataset

Unlike part1, this section creates datasets with a noise percentage of 0 percent, where database_choose and h_data_choose are used for training, and database_validate and h_data_validate are used for validation

In [ ]:
pinn_config = config["pinn"]
pinn_dataset_config = pinn_config["dataset"]
model_config = config["model"]
summary_config = config["summary"]
optimizer_config = config["optimizer"]
epochs = optimizer_config["epochs"]


# create dataset for training and validating
pinn_dataset = create_pinn_dataset(
    case_name, pinn_dataset_config)
database_choose, h_data_choose, database_validate, h_data_validate = pinn_dataset

## Load model

What structural model is used in part1, where you need to use the model of the corresponding structure and load the saved model

In [ ]:
model = MultiScaleFCSequential(in_channels=model_config["in_channels"],
                               out_channels=model_config["out_channels"],
                               layers=model_config["layers"],
                               neurons=model_config["neurons"],
                               residual=model_config["residual"],
                               act=model_config["activation"],
                               num_scales=1)
# load checkpoint
ckpt_name = f"{case_name}_nn-{epochs + 1}.ckpt"
ckpt_path = summary_config["save_checkpoint_epochs"]
model_dict = load_checkpoint(os.path.join(
    ckpt_path, ckpt_name))
load_param_into_net(model, model_dict)

## Build potential terms

After obtaining the result of genetic algorithm in part1, the candidate items of the equation are obtained, and the corresponding Dict_name, Dict_n, Dict are modified, and the code of term calculation in util.py in src directory is modified according to its calculation requirements

In [ ]:
# result from ga
# need to be change by hand
dict_name = get_dict_name(case_name)

optimizer = nn.Adam(model.trainable_params(),
                    optimizer_config["initial_lr"])
# set ascend
if use_ascend:
    loss_scaler = DynamicLossScaler(1024, 2, 100)
    auto_mixed_precision(model, model_config["amp_level"])
else:
    loss_scaler = None


def forward_fn(dataset, lefts, coef_list, dict_name, terms_dict):
    database_choose, h_data_choose = dataset
    prediction = model(database_choose)
    f1 = nn.MSELoss(reduction='mean')(prediction, h_data_choose)
    loss = pinn_loss_func(f1, lefts, coef_list,
                          dict_name, terms_dict)

    if use_ascend:
        loss = loss_scaler.scale(loss)
    return loss


grad_fn = value_and_grad(
    forward_fn, None, optimizer.parameters, has_aux=False)


@jit
def train_step(dataset, lefts, coef_list, dict_name, terms_dict):
    loss, grads = grad_fn(dataset, lefts, coef_list, dict_name, terms_dict)
    if use_ascend:
        loss = loss_scaler.unscale(loss)
    loss = ops.depend(loss, optimizer(grads))
    return loss

## Start training

Define the trainer and start training

In [ ]:
for epoch in range(1, epochs + 1):
    time_beg = time.time()
    prediction = model(database_choose)
    grads, libraries = cal_grads(case_name, model, database_choose,
                                 pinn_dataset_config["choose_train"])

    terms = cal_terms(case_name, prediction, grads)

    # terms in numpy
    terms_dict, dict_n = get_dicts(terms)

    libraries = update_lib(case_name, dict_name, libraries, dict_n)

    # Lasso
    lefts = get_lefts(case_name, grads, prediction, False)
    coef_list, lst_list = calculate_coef(lefts, libraries, epoch, config)

    model.set_train(True)

    # train step
    lefts = get_lefts(case_name, grads, True)

    step_train_loss = train_step((database_choose, h_data_choose),
                                 lefts, coef_list, dict_name, terms_dict)
    # set model to eval mode
    model.set_train(False)

    # put zeros
    if epoch >= 1000:
        for lst in lst_list:
            for i in range(lst.shape[0]):
                if np.abs(lst[i]) < pinn_config["kesi"]:
                    dict_name.pop(i)
                    break

    if epoch % summary_config["validate_interval_epochs"] == 0:
        print("Dict_Name %s", dict_name)
        # current epoch loss
        print("epoch: %s train loss: %s epoch time: %.3fms",
              epoch, step_train_loss, (time.time() - time_beg) * 1000)
        evaluate(model, database_validate, h_data_validate, config)

    # save checkpoint
    if epoch % summary_config["save_checkpoint_epochs"] == 0:
        ckpt_name = f"{case_name}_pinn-{epoch + 1}.ckpt"
        save_checkpoint(model, os.path.join(
            ckpt_path, ckpt_name))

...
Theta:[-1.0167898e-10 -3.5906428e-01  3.8740728e-03  6.8805676e-08-2.0166181e-05 -6.6712929e-04]Dict_Name [1, 2, 3, 4, 5, 6]epoch: 100 train loss: 0.16274601 epoch time: 7609.529ms
    predict total time: 2.933502197265625 ms
    l2_error:  0.03319109569478303
Theta:[-6.1833136e-11 -3.9782286e-01  3.7144462e-03  4.6671396e-08-1.5555745e-05 -9.6872513e-04]Dict_Name [1, 2, 3, 4, 5, 6]epoch: 200 train loss: 0.12895139  epoch time: 8463.217ms
    predict total time: 2.847433090209961 ms
    l2_error:  0.3844950973176981
...
Theta:[[-0.4369226 ][ 0.00145796][ 0.00145796]]Dict_Name [2, 3, 6]epoch: 2390 train loss: 0.08259561 epoch time: 14893.079ms
    predict total time: 1.985311508178711 ms
    l2_error:  0.0062716909767681125
Theta:[[-0.4367841 ][ 0.00145637][ 0.00145637]]Dict_Name [2, 3, 6]epoch: 2400 train loss: 0.08224905 epoch time: 15054.685ms
    predict total time:  3.087759017944336 ms
    l2_error:  0.006187851509927684
Theta:[[-0.43811008][ 0.00146007][ 0.00146007]]Dict_Name

![visual_burgers](./images/burgers.jpg)